In [4]:
# Colab/Jupyter: installa dipendenze
#!pip install pytorch-grad-cam opencv-python pillow tqdm
!pip install -qq grad-cam==1.4.6 torchinfo==1.7.1 #torch==1.12.1 torchvision==0.13.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
import torch.nn as nn
from torchvision.models import resnet18

# === 1) CONFIGURAZIONE ===
INP_DIR = "/content/input"
OUT_DIR    = "/content/output"
MODEL_PATH = "/content/predictors.pth"

os.makedirs(OUT_DIR, exist_ok=True)

IMG_SIZE   = 224  # <-- ADATTA alle tue dimensioni di training
#MEAN       = [0.485, 0.456, 0.406]   # <-- ADATTA se diverso in training
#STD        = [0.229, 0.224, 0.225]

MEAN = [0.5]   # <-- usa i valori del training se li conosci
STD  = [0.5]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# === 2) CARICA IL MODELLO ===
# ESEMPIO: se avevi salvato solo state_dict su una ResNet18

from torchvision.models import resnet18

model = resnet18(num_classes=2)  # <-- ADATTA il numero classi
# 1) porta la prima conv a 1 canale
model.conv1 = nn.Conv2d(
    in_channels=1, out_channels=64,
    kernel_size=7, stride=2, padding=3, bias=False
)

state = torch.load(MODEL_PATH, map_location="cpu")
# Se salvato con keys 'state_dict' o simili, estrai la parte giusta
if isinstance(state, dict) and "state_dict" in state:
    state = {k.replace("model.", "").replace("module.", ""): v for k, v in state["state_dict"].items()}
elif isinstance(state, dict):
    state = {k.replace("module.", ""): v for k, v in state.items()}


model.load_state_dict(state, strict=False)
model.to(DEVICE).eval()

# === 3) SCEGLI IL LAYER TARGET PER GRAD-CAM ===
# ResNet: ultimo blocco conv
target_layer = model.layer4[-1]

# === 4) PREPROCESS ===
preprocess = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),                     # da immagine L -> tensor [1,H,W]
    transforms.Normalize(mean=MEAN, std=STD),
])

# === 5) GRAD-CAM ===
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

def load_image_RGB(path):
    img = Image.open(path).convert("RGB")   # se grayscale: .convert("L") e poi .convert("RGB") oppure gestisci 1 canale
    img_np = np.array(img)
    return img, img_np

def load_image(path):
    # input per il modello: 1 canale
    img = Image.open(path).convert("L")         # grayscale
    gray_np = np.array(img)                     # [H,W], uint8

    # per l’overlay (serve 3 canali in [0,1] per show_cam_on_image)
    rgb_for_overlay = np.stack([gray_np, gray_np, gray_np], axis=-1)
    return img, rgb_for_overlay

def to_tensor(img_pil):
    return preprocess(img_pil).unsqueeze(0).to(DEVICE)

def run_cam_on_image(img_pil, rgb_np, target_category=None):
    input_tensor = to_tensor(img_pil)

    with torch.enable_grad():
        with GradCAM(model=model, target_layers=[target_layer], use_cuda=(DEVICE=="cuda")) as cam:
            targets = None
            if target_category is not None:
                targets = [ClassifierOutputTarget(int(target_category))]
            grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]  # [Hc, Wc] = [IMG_SIZE, IMG_SIZE]

    # >>> resize CAM to original image size <<<
    H, W = rgb_np.shape[:2]
    cam_resized = cv2.resize(grayscale_cam, (W, H), interpolation=cv2.INTER_LINEAR)

    rgb_float = rgb_np.astype(np.float32) / 255.0
    visualization = show_cam_on_image(rgb_float, cam_resized, use_rgb=True)
    return cam_resized, visualization

# === 6) LOOP SULLE IMMAGINI ===
with torch.no_grad():
  for fname in tqdm(sorted(os.listdir(INP_DIR))):
      if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp", ".webp")):
          continue

      fpath = os.path.join(INP_DIR, fname)
      img_pil, rgb_np = load_image(fpath)

      # 1) Predizione (senza grad)
      with torch.no_grad():
          tensor = to_tensor(img_pil)
          logits = model(tensor)
          pred_idx = int(torch.argmax(logits, dim=1).item())

      # 2) Grad-CAM (con grad abilitato!)
      #   - in caso qualche cella abbia disabilitato globalmente i grad,
      #     forziamo l'abilitazione con enable_grad()
      with torch.enable_grad():
          grayscale_cam, overlay = run_cam_on_image(img_pil, rgb_np, target_category=pred_idx)

      # 3) Salvataggi
      base, ext = os.path.splitext(fname)
      heatmap_path = os.path.join(OUT_DIR, f"{base}_cam.png")
      overlay_path = os.path.join(OUT_DIR, f"{base}_overlay.png")

      cv2.imwrite(heatmap_path, (grayscale_cam * 255).astype(np.uint8))
      cv2.imwrite(overlay_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))


100%|██████████| 2/2 [00:00<00:00,  2.62it/s]
